In [5]:
import pandas as pd 
from path_fns.filepaths import TRANSFORMED_MASTER_DATA_ONE_ROW_PER_PERSON
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 20

### Person data

In [6]:
df_persons = pd.read_parquet(TRANSFORMED_MASTER_DATA_ONE_ROW_PER_PERSON)


In [11]:
df_persons


,human,dod,family_name,dob,given_name,country_citizen,occupation,humanLabel,given_nameLabel,family_nameLabel,occupationLabel,country_citizenLabel,sex_or_genderLabel,place_birth,birth_country,place_birthLabel,birth_countryLabel,birth_name,humanDescription,name_native_language,humanAltLabel,residence,residenceLabel,residence_countryLabel,pseudonym,ethnicity,ethnicityLabel,full_name_arr,birth_coordinates,residence_coordinates
0,Q6617494,[1953-01-01],[Q2339506],[1870-07-26],[Q18201513],[Q30],"[Q1028181, Q10732476]",[Sarah Stein],[Sarah],[Stein],"[art collector, painter]",[United States of America],[female],[Q62],[Q30],[San Francisco],[United States of America],[],[American art collector and painter (1870-1953)],[],[Sarah Samuels Stein],"[Q90, Q246199]","[Paris, Garches]",[France],[],[],[],"[Sarah Samuels Stein, Sarah Stein]","[{'lat': 37.7775, 'lng': -122.416388888}]","[{'lat': 48.856944444, 'lng': 2.351388888}, {'..."
1,Q1677724,"[1953-01-01, 1953-09-24]","[Q76416996, Q23041619]",[1878-10-17],[Q19688634],[Q29],"[Q193391, Q82955]","[Jacobo Fitz-James Stuart, 17th Duke of Alba]",[Jacobo],"[Fitz-James, Stuart]","[politician, diplomat]",[Spain],[male],[Q2807],[Q29],[Madrid],[Spain],[Jacobo Fitz-James Stuart y Falcó],[Spanish noble],[Jacobo Fitz-James Stuart y Falcó],[Jacobo Stuart Fitz James Falco Portocarrero y...,[Q3288223],[Liria Palace],[Spain],[],[],[],[Jacobo Stuart Fitz James Falco Portocarrero y...,"[{'lat': 40.416666666, 'lng': -3.7025}]","[{'lat': 40.4277, 'lng': -3.7124}]"
2,Q11936206,[1953-01-01],[Q105359098],[1900-01-01],[Q16282809],[Q29],[Q33999],[Matilde Xatart i Solà],[Matilde],[Xatart],[actor],[Spain],[female],[Q1492],[Q29],[Barcelona],[Spain],[Matilde Xatart i Solà],[Spanish actor],[],[],[Q2887],[Santiago],[Chile],[],[],[],"[Matilde Xatart, Matilde Xatart i Solà]","[{'lat': 41.3825, 'lng': 2.176944444}]","[{'lat': -33.45, 'lng': -70.666666666}]"
3,Q5891949,[1953-01-01],[Q16875352],[1894-01-01],[Q632842],[Q183],[Q2374149],[Hans Teschner],[Hans],[Teschner],[botanist],[Germany],[male],[Q64],[Q183],[Berlin],[Germany],[],[botanist (1894-1953)],[],[Teschner],[Q183],[Germany],[Germany],[],[],[],"[Teschner, Hans Teschner]","[{'lat': 52.516666666, 'lng': 13.383333333}]","[{'lat': 51.0, 'lng': 10.0}]"
4,Q855,[1953-03-05],[Q22179713],[1878-12-18],[Q29051339],"[Q34266, Q2305208, Q15180]","[Q6051619, Q3242115, Q49757, Q82955]",[Joseph Stalin],[Ioseb],[Stalin],"[opinion journalist, revolutionary, politician...","[Russian Socialist Federative Soviet Republic,...",[male],[Q19583],[Q230],[Gori],[Georgia],[იოსებ ბესარიონის ძე ჯუღაშვილი],[leader of the Soviet Union from 1924 to 1953],"[Иосиф Виссарионович Джугашвили, იოსებ ბესარიო...","[Stalin, Koba, Uncle Joe, Ioseb Besarionis dze...","[Q9248, Q155243, Q196388, Q1020157, Q4322358, ...","[Saint Petersburg, Baku, Turukhansk, Narym, No...","[Russia, Azerbaijan]",[Сталин],"[Q106974, Q18973]","[Ossetians, Georgians]","[Сталин, Ioseb Dzhugashvili, Stalin, Josef Sta...","[{'lat': 41.981686111, 'lng': 44.112416666}]","[{'lat': 59.95, 'lng': 30.316666666}, {'lat': ..."
5,Q2982529,[1953-03-17],[Q65115026],[1874-07-28],[Q16275223],[Q142],[Q36180],[Colette Yver],[Colette],[Yver],[writer],[France],[female],[Q549330],[Q142],[Segré],[France],[Antoinette de Bergevin],[French writer],[Colette Yver],[],[Q30974],[Rouen],[France],[],[],[],[Colette Yver],"[{'lat': 47.686388888, 'lng': -0.8725}]","[{'lat': 49.443055555, 'lng': 1.1025}]"
6,Q3028585,[1953-03-21],[Q2588593],[1888-09-18],[Q1929038],[Q39],[Q212980],[Toni Wolff],[Toni],[Wolff],[psychologist],[Switzerland],[female],[Q72],[Q39],[Zürich],[Switzerland],[],[psychologist (1888-1953)],[],[Toni Anna Wolff],[Q39],[Switzerland],[Switzerland],[],[],[],"[Toni Anna Wolff, Toni Wolff]","[{'lat': 47.374444444, 'lng': 8.541111111}]","[{'lat': 46.798562, 'lng': 8.231973}]"
7,Q85917998,[1953-03-23],[Q10368463],[1878-07-07],[Q16799105],[Q45],"[Q1371378, Q189290, Q36180]",[Alberto Artur Sarmento],[Alberto],[Sarmento],"[ethnologist, military officer, writer]",[Port

In [ ]:

for col in ["humanlabel", "humanaltlabel", "birth_name", "birth_name", "given_namelabel", "family_namelabel"]:
    df_person = df_person.withColumn(col, expr(f'lower({col})'))

df_person.createOrReplaceTempView("df_person")
# spark.sql("select * from df_person where human = 'Q38082'").toPandas() lewis caroll
df_person.limit(2).toPandas()

### Names

In [ ]:
from master_data.names import get_df_given_names_with_freqs, get_df_family_names_with_freqs
df_given_names = get_df_given_names_with_freqs(spark)
df_family_names = get_df_family_names_with_freqs(spark)
df_given_names.createOrReplaceTempView("df_given_names")
df_family_names.createOrReplaceTempView("df_family_names")
display(df_given_names.limit(2).toPandas())
display(df_family_names.limit(2).toPandas())

### Person postcode lookup

In [ ]:
from master_data.utils import get_person_nearby_postcodes_lookup
df_point_postcode = get_person_nearby_postcodes_lookup(spark)
df_point_postcode.createOrReplaceTempView("df_point_postcode")


df_point_postcode.limit(3).toPandas()

## Occupations

In [ ]:
occupations = spark.read.parquet("scrape_wikidata/raw_data/occupations/")
occupations.createOrReplaceTempView("occupations")
occupations.limit(2).toPandas()

sql = """
select human, array_distinct(collect_list(occupationLabel)) as occupation_options
from occupations
group by human
"""

occupations = spark.sql(sql)
occupations.createOrReplaceTempView("occupations")
occupations.limit(2).toPandas()

## Master data

#### Get list of given names and family names

In [ ]:
from master_data.names import get_df_filter, name_split
df_person = name_split(df_person, spark)
df_person.limit(2).toPandas()

In [ ]:
remove_dupe_country_citizen = """
array_distinct(
    transform(
        split(h.country_citizenlabel, ' \\\\| '), 
            x -> case 
                    when x = 'United Kingdom of Great Britain and Ireland' then 'United Kingdom' 
                    else x 
                    end
    )
)
"""

sql = f"""
select 
h.row_num,
h.human, 
h.humanlabel,
split(h.humanaltlabel, ", ") as humanaltlabel,
occ.occupation_options,
substr(h.dob,1,10) as dob, 


{remove_dupe_country_citizen}  as country_citizenship,


place_birthlabel as birth_place,
birth_countrylabel as birth_country,
sex_or_genderlabel as gender,



residencelabel as residence_place,
residence_countrylabel as residence_country,

pc.birthplace_nearby_locs,
pc.residence_nearby_locs,
pc.random_nearby_locs,

h.given_name_1 as given_name_1,
n1.alt_names as alt_given_name_1,
h.given_name_2 as given_name_2,
n2.alt_names as alt_given_name_2,
h.given_name_3 as given_name_3,
n3.alt_names as alt_given_name_3,
h.family_name_1 as family_name_1,
n4.alt_names as alt_family_name_1,
h.family_name_2 as family_name_2,
n5.alt_names as alt_family_name_2

from df_person as h

left join df_given_names as n1
on lower(h.given_name_1) = n1.original_name

left join df_given_names as n2
on lower(h.given_name_2) = n2.original_name

left join df_given_names as n3
on lower(h.given_name_3) = n3.original_name

left join df_family_names as n4
on lower(h.family_name_1) = n4.original_name

left join df_family_names as n5
on lower(h.family_name_2) = n5.original_name

left join df_point_postcode as pc
on h.human = pc.person

left join occupations  as occ
on h.human = occ.human



"""
df_final = spark.sql(sql)
df_final.createOrReplaceTempView("df_final")  


df_final.limit(10).toPandas()


## Now want to link parent back onto this table, and their location.

In [ ]:
df_family = spark.read.parquet("scrape_wikidata/raw_data/family/parent_child/")
 
df_one_parent = df_family.dropDuplicates(["child"])
# df_one_child = df_family.dropDuplicates(["human"])
df_one_parent.createOrReplaceTempView("df_one_parent") 
# df_one_child.createOrReplaceTempView("df_one_child") 
df_family.limit(2).toPandas()

In [ ]:
sql = """

select 
    df.*, 
    p.human as parent, 
    p_details.birthplace_nearby_locs as parent_birthplace_loc, 
    p_details.random_nearby_locs as parent_random_loc


from df_final as df
left join df_one_parent as p
on df.human = p.child
left join df_final as p_details
on p.human = p_details.human




"""
# No need for child as if parents take childs details and children take parents, then you don't achieve parents and children having the same address!
# left join df_one_child as c
# on df.human = c.human
# left join df_final as c_details
# on c.child = c_details.human
df_final_with_parents = spark.sql(sql)

In [ ]:
df_final_with_parents = df_final_with_parents.repartition(100)
df_final_with_parents.write.mode('overwrite').parquet("scrape_wikidata/clean_data/master_data/")

In [ ]:
import glob
import os 
files = sorted(
    glob.glob("scrape_wikidata/clean_data/master_data/*.parquet")
)

for i, f in enumerate(files):
    
    base = os.path.basename(f)
    f2 = f.replace(base, f"{i:03}.parquet")
    
    os.rename(f, f2)
    
files = sorted(
    glob.glob("scrape_wikidata/clean_data/master_data/.*crc")
)
for f in files:
    os.remove(f)

In [ ]:
spark.read.parquet("corrupted_data/uk_citizens_groupsize_20/").count()